In [1]:
import os
import sys
sys.path.insert(0, os.path.realpath('__file__'))

import numpy as np
import matplotlib.pyplot as plt
import pickle
import pyidi

c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Load the video

In [2]:
filename = 'data/data_synthetic.cih'
video = pyidi.pyIDI(filename)

Video info can easily be accessed

In [3]:
video.info

{'Record Rate(fps)': 1,
 'Shutter Speed(s)': 1.0,
 'Total Frame': 101,
 'Original Total Frame': 101,
 'Start Frame': 0,
 'Image Width': 256,
 'Image Height': 128,
 'Color Type': 'Mono',
 'Color Bit': 16,
 'File Format': 'MRaw',
 'EffectiveBit Depth': 12,
 'EffectiveBit Side': 'Lower',
 'Comment Text': 'Generated sequence.<br>Modify measurement info in<br>created .cih file if necessary.'}

In [4]:
%matplotlib qt
import matplotlib.animation as animation

# Define the update function for the animation
def update(frame):
    im.set_array(video.mraw[frame])
    return im,

# Create a figure and axis for the animation
fig, ax = plt.subplots()
im = ax.imshow(video.mraw[0], cmap='gray')

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=range(1000), interval=200, blit=True)

# Display the animation
plt.show()


Assign the `Lucas Kanade` method

In [5]:
%matplotlib qt
tol = 1e-4
video = pyidi.pyIDI(filename)
video.set_method('lk')
video.method.configure(reference_image = (0,100), resume_analysis = False, tol=tol)
reference_image = video.method._set_reference_image(video, video.method.reference_image)
Gi, Gj = np.gradient(reference_image)
G = np.sqrt(Gi**2 + Gj**2)
highest_points = np.argpartition(G.flatten(), -10)[-10:]
arguments = np.unravel_index(highest_points, G.shape)

fig, ax = plt.subplots()
ax.scatter(arguments[1], arguments[0])
ax.imshow(reference_image, cmap='gray')
plt.show()
video.set_points(np.array(arguments).T)
video.show_points()
displacementes_2d = video.get_displacements(processes=1, autosave=False)
video.method.total_steps

--- Starting new analysis ---
 
Interpolating the reference image...
...done in 0.01 s


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


VBox()

Time to complete: 0.5 s


4220

Identify the displacements using 2 processes

In [6]:
%matplotlib qt
video = pyidi.pyIDI(filename)
video.set_method('lk_1D')
d = (0.4, 0.6)
video.method.configure(reference_image = (0,100), d=d, resume_analysis = False, tol=tol)
reference_image = video.method._set_reference_image(video, video.method.reference_image)
Gi, Gj = np.gradient(reference_image)
G = np.sqrt(Gi**2 + Gj**2)
highest_points = np.argpartition(G.flatten(), -10)[-10:]
arguments = np.unravel_index(highest_points, G.shape)

fig, ax = plt.subplots()
ax.scatter(arguments[1], arguments[0])
ax.imshow(reference_image, cmap='gray')
plt.show()
video.set_points(np.array(arguments).T)
video.show_points()
video.method.d, video.method.ex, video.method.ey

(array([0.5547002 , 0.83205029]), 0.5547001962252291, 0.8320502943378436)

In [7]:
displacements = video.get_displacements(processes=1, autosave=False)
video.method.total_steps

--- Starting new analysis ---
 
Interpolating the reference image...
...done in 0.01 s


VBox()

Time to complete: 1.1 s


18960

An existing analysis can also be loaded without computation and setting the arguments.

In [8]:
t_vec = np.arange(0, len(displacements[0])) * video.info['Record Rate(fps)']

fig, ax = plt.subplots()
ax.set_xlabel('Time (s)')
ax.set_ylabel('Displacement (pixels)')
for point in displacements:
    # ax.plot(t_vec, point[:,0],'b:')
    # ax.plot(t_vec, point[:,1],'b--')
    ax.plot(t_vec, point[:,0]*d[0] + point[:,1]*d[1],'b', label='1D, d = (0.55, 0.83)')
for point in displacementes_2d:
    ax.plot(t_vec, point[:,0]*d[0] + point[:,1]*d[1], 'r', label='2D, d = (0.55, 0.83)')
ax.plot(t_vec, displacementes_2d[0, :,0], 'r:', label = '2D - x', alpha=0.5)
ax.plot(t_vec, displacementes_2d[0, :,1], 'r--', label = '2D - y', alpha=0.5)
# ax.plot(t_vec, np.arctan2(displacementes_2d[0, :,1], displacementes_2d[0, :,0]), 'r--', label = 'direction', alpha=0.5)
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys())
plt.show()